<a href="https://colab.research.google.com/github/SuvitKumar003/MINI-PROJECTS-ML/blob/main/optimizedCNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install keras-tuner

In [16]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [17]:
print(tf.__version__)

2.15.0


In [18]:
fashion_mnist=keras.datasets.fashion_mnist

In [20]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [21]:
train_images=train_images/255.0
test_images=test_images/255.0

In [22]:
train_images[0].shape

(28, 28)

In [24]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [25]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [26]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [27]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [28]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 29s]
val_accuracy: 0.9100000262260437

Best val_accuracy So Far: 0.9100000262260437
Total elapsed time: 00h 03m 20s


In [29]:
model=tuner_search.get_best_models(num_models=1)[0]

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 80)        800       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        46144     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 48)                1769520   
                                                                 
 dense_1 (Dense)             (None, 10)                490       
                                                                 
Total params: 1816954 (6.93 MB)
Trainable params: 1816954 (6.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=2)

Epoch 3/10
1688/1688 [==============================] - 10s 5ms/step - loss: 0.1406 - accuracy: 0.9475 - val_loss: 0.2559 - val_accuracy: 0.9110
Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1003 - accuracy: 0.9630 - val_loss: 0.2822 - val_accuracy: 0.9137
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0730 - accuracy: 0.9734 - val_loss: 0.3321 - val_accuracy: 0.9130
Epoch 6/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0514 - accuracy: 0.9819 - val_loss: 0.3446 - val_accuracy: 0.9155
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0376 - accuracy: 0.9861 - val_loss: 0.4355 - val_accuracy: 0.9128
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0308 - accuracy: 0.9889 - val_loss: 0.4619 - val_accuracy: 0.9093
Epoch 9/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0272 - accuracy: 0.9904 - val_loss: 0.5281 - val_accurac